In [54]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
import time

In [19]:
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
driver = webdriver.Firefox(options=options)

In [20]:
URL_BASE = 'https://www.google.com/search?q='
CODIGOS_DESCRIPCION_PATH = 'cod_desc_data.csv'

In [21]:
cod_desc_data = pd.read_csv(CODIGOS_DESCRIPCION_PATH)

In [39]:
n_lotes = 20
lote_size = int(len(cod_desc_data) / n_lotes)

In [53]:
# OBTENER LOTES (BATCHES)
lotes = []
for i in range(n_lotes):
    if i < n_lotes - 1:
        lote = cod_desc_data.loc[i*lote_size: (i+1) * lote_size]
    else:
        lote = cod_desc_data.loc[i*lote_size:]
    lotes.append(lote)

In [55]:
# BUSQUEDA
resultados_busqueda = []
for lote in lotes:
    for descripcion in tqdm(lote.descripcion):
        driver.get(URL_BASE + descripcion + ' symptoms')
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        res = soup.findAll(class_ = 'yuRUbf')
        try:
            resultados_busqueda.append(list(res[0].children)[0].get('href'))
        except: 
            resultados_busqueda.append("Error de busqueda")
    time.sleep(2*60)

 24%|███████████████████▎                                                            | 148/613 [06:40<20:57,  2.70s/it]


TimeoutException: Message: TimedPromise timed out after 300000 ms
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.jsm:472:5
bail@chrome://remote/content/marionette/sync.js:231:19


In [ ]:
resultados_busqueda

In [ ]:
cod_desc_url = pd.concat([cod_desc_data, pd.Series(resultados_busqueda)], axis=1)
cod_desc_url.columns = ['codigo', 'descripcion', 'url']

In [ ]:
cod_desc_url.to_csv('cod_desc_url.csv', index=False)